In [1]:
import json
from monty.json import MontyDecoder,MontyEncoder
import numpy as np
from pymatgen.io.ase import AseAtomsAdaptor
from matplotlib import pyplot as plt

with open("Ti_data.json") as f:
    data_collection=json.loads(f.read(),cls=MontyDecoder)

In [2]:
import numpy as np
vacancies_ideal_id={}
for i in np.arange(len(data_collection)):
    if data_collection[i]['metadata']['perturbation']=='vacancies_ideal':
        vacancies_ideal_id[data_collection[i]['metadata']['proto']]=i

vacancies_id=[]
for i in np.arange(len(data_collection)):
    if data_collection[i]['metadata']['perturbation']=='vacancies':
        vacancies_id.append(i)

In [3]:
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from ase.neighborlist import neighbor_list
from collections import Counter

ideallist=list(vacancies_ideal_id.keys())
ideal=ideallist[0]

atoms=AseAtomsAdaptor.get_atoms(data_collection[vacancies_ideal_id[ideal]]['structure'])
nl=neighbor_list('ijd',atoms,cutoff=5)


In [4]:
for id in vacancies_id:
    sg=SpacegroupAnalyzer(data_collection[id]['structure'])
    sgd=sg.get_symmetrized_structure().as_dict()
    cnt=Counter(sgd['equivalent_positions'])

In [5]:
from pymatgen.io.vasp.inputs import Poscar
from ase.io import write
casi=Poscar.from_file('./ti_ideal_va/casi-alth/POS')
mp_865373=Poscar.from_file('./ti_ideal_va/mp-865373POSCAR/POS')

In [6]:
sg=SpacegroupAnalyzer(casi.structure)
sgd=sg.get_symmetrized_structure().as_dict()
cnt_casi=Counter(sgd['equivalent_positions'])
print(len(cnt_casi))

2


In [7]:
sg=SpacegroupAnalyzer(mp_865373.structure)
sgd=sg.get_symmetrized_structure().as_dict()
cnt_mp865373=Counter(sgd['equivalent_positions'])
print(len(cnt_mp865373))

2


In [12]:
def gen_halfway_struct(name,structure,cutoff=3.5):

    sg=SpacegroupAnalyzer(structure)
    sgd=sg.get_symmetrized_structure().as_dict()
    cnt=Counter(sgd['equivalent_positions'])
    sites=cnt.keys()
    atoms=AseAtomsAdaptor.get_atoms(structure)
    nl=neighbor_list('ijd',atoms,cutoff=cutoff)
    for site in sites:
        nl[0]==site
    sg=SpacegroupAnalyzer(structure)
    sgd=sg.get_symmetrized_structure().as_dict()
    cnt=Counter(sgd['equivalent_positions'])
    asym_sites=list(cnt.keys())
    atoms=AseAtomsAdaptor.get_atoms(structure)
    nl=neighbor_list('ijd',atoms,cutoff=cutoff)
    for site in asym_sites:
        plt.figure()
        nlpos=np.where(nl[0]==site)[0]
        neighequ_species=sgd['equivalent_positions'][nl[1][nlpos]]
        for i in np.arange(len(nlpos)):
            species_=[22]*len(structure.species)
            species_[site]=29
            whichsites=np.where(neighequ_species==neighequ_species[i])[0]
            findsite=whichsites[np.where(whichsites!=site)[0][0]]
            species_[findsite]=29
            new_atoms=atoms
            new_atoms.set_atomic_numbers(species_)
            write('./VacancyPath/'+name+'_'+str(site)+'_'+str(neighequ_species[i])+'.vasp',new_atoms)
            plt.scatter(neighequ_species[i],nl[2][nlpos][i])
        plt.savefig('./VacancyPath/'+name+'_site'+str(site)+'.png')
        plt.close()
        


In [13]:
for id in vacancies_id:
    if data_collection[id]['calc']=='final':
        if data_collection[id]['metadata']['proto']=='casi-alth':
            assert data_collection[id]['metadata']['deleted_site'] in cnt_casi.keys()
            gen_halfway_struct('casi',casi.structure)
        elif data_collection[id]['metadata']['proto']=='mp-865373POSCAR':
            assert data_collection[id]['metadata']['deleted_site'] in cnt_mp865373.keys()
            gen_halfway_struct('mp_865373',mp_865373.structure)
        else:
            idealid=vacancies_ideal_id[data_collection[id]['metadata']['proto']]
            sg=SpacegroupAnalyzer(data_collection[idealid]['structure'])
            sgd=sg.get_symmetrized_structure().as_dict()
            cnt=Counter(sgd['equivalent_positions'])
            assert data_collection[id]['metadata']['deleted_site'] in cnt.keys()
            gen_halfway_struct(data_collection[id]['metadata']['proto'],data_collection[idealid]['structure'])
